In [27]:
from dotenv import load_dotenv
import os
import weaviate
from weaviate.classes.init import Auth

load_dotenv()
WEAVIATE_URL = "https://konvsbviroeqczjr3hsitw.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(api_key=WEAVIATE_API_KEY),
)
print(client.is_ready())
client.close()

True


In [50]:
from IPython.display import display, Markdown

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [44]:
from langchain_community.document_loaders import PyPDFLoader

pdf_folder = "C:/Users/csing/VSCode/Projects/RAG_LangChain_MistralAI_Weaviate/data"
documents = []

for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        loader = PyPDFLoader(pdf_path, extract_images=True)
        documents.extend(loader.load())

In [ ]:
print(documents[0].page_content)

In [45]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(documents)